In [ ]:
from abc import ABC, abstractmethod
import numpy as np
from itertools import product
import numpy as np
#from derl.common import flat_layers_iterator#??? """ Iterator over all layers of a given model. """
import gym
import torch
from torch.nn import Module
from itertools import chain
from torch.nn import init
import math

## Device

In [ ]:
device = 'cuda:3'

## Space

In [ ]:
from my_base import Space 
from selection.spaces import MultiDiscrete, SkipSpace, ChooseSpace, SearchSpace

## Select Layers

In [ ]:
from selection.select_layers import SelectLayer, SelectModelFromLayers, reset_parameters, instantiate_model

# Get model

In [ ]:
from selection.custom_layers import Conv2dSame, MaxPool2dSame, MaybeRescale, pad_to_size, PadToSize, PadFeatureMaps

In [ ]:
from selection.base_layers import impala_base_layers
def get_random_model(env, device):
    """ Returns a randomly sampled model from the architecture space"""
    model = SelectModelFromLayers(output_feats = [env.action_space.n, 1], base_layers=impala_base_layers())
    sample = model.space.sample()
    print(sample)
    #if logdir is not None:
        #with open(os.path.join(logdir, "sample.txt"), "a") as samplefile:
        #      samplefile.write(' '.join(map(str, sample.numpy())) + "\n")
    model.select(sample)
    x = torch.rand(1, *env.observation_space.shape).to(device)
    instantiate_model(model.to(device), x)
    
    return model

## Train RL models

In [ ]:
from learners.learners import PPOLearner

In [ ]:
#from derl.env import make as make_env
import runners.my_runners

In [ ]:
from env.my_envs import make

In [ ]:

env = make(env_id = 'FreewayNoFrameskip-v0', nenvs = 8)
model = get_random_model(env, device=None)

In [ ]:
args = PPOLearner.get_defaults(env_type = 'atari')
learner = PPOLearner.from_env_args(env, args, device=None, model=model)

In [ ]:
args

In [ ]:
#3 loss - 1 rew
learner.learn(args['num_train_steps'], 'new_weights/nas_found_8envs.pth', 1)

# ENAS

In [ ]:
args = None
if args is None:
    args = {
            "atari": {
                "num_train_steps": 10e6,
                "nenvs": 8,
                "num_runner_steps": 128,
                "gamma": 0.99,
                "lambda_": 0.95,
                "num_epochs": 3,
                "num_minibatches": 4,
                "cliprange": 0.1,
                "value_loss_coef": 0.25,
                "entropy_coef": 0.01,
                "max_grad_norm": 0.5,
                "lr": 2.5e-4,
                "optimizer_epsilon": 1e-5,
                "search_space": "impala",
                "num_learner_steps":10,
                "num_nas_runner_steps":3,
                "nas_lr": 1e-3,
                "nas_entropy_coef": 1e-4,
                "nas_optimizer_epsilon":1e-5,
                "nas_baseline_momentum":0.2,
                }}
args = args['atari']

In [ ]:
from train_nas import train
controller, learner = train(device, args, 'FreewayNoFrameskip-v0', logdir = '/gpfs/gpfs0/n.mazyavkina/new_weights/env81')

In [ ]:
learner.select_model.selection